This notebook prepared based on the framework developed by <cite id="imgp5"><a href="#zotero%7C12028432%2FPN66USTB">(Chukalla et al., 2022)</a></cite>. 

## Important References

* <!-- BIBLIOGRAPHY START -->
<div class="csl-bib-body">
  <div class="csl-entry"><i id="zotero|12028432/KW9EFC6I"></i>Bashe, T., Alamirew, T., &#38; Dejen, Z. A. (2023). Evaluating Performance of Community-based Irrigation Schemes Using Remote-sensing Technologies to Enhance Sustainable Irrigation Water Management. <i>Water Conservation Science and Engineering</i>, <i>8</i>(1), 45. <a href="https://doi.org/10.1007/s41101-023-00222-y">https://doi.org/10.1007/s41101-023-00222-y</a></div>
* <div class="csl-entry"><i id="zotero|12028432/QX6S99F9"></i>Bastiaanssen, W. G. M., &#38; Bos, M. G. (1999). Irrigation Performance Indicators Based on Remotely Sensed Data: a Review of Literature. <i>Irrigation and Drainage Systems</i>. <a href="https://doi.org/10.1023/a:1006355315251">https://doi.org/10.1023/a:1006355315251</a></div>
* <div class="csl-entry"><i id="zotero|12028432/PN66USTB"></i>Chukalla, A. D., Mul, M. L., van der Zaag, P., van Halsema, G., Mubaya, E., Muchanga, E., den Besten, N., &#38; Karimi, P. (2022). A framework for irrigation performance assessment using WaPOR data: the case of a sugarcane estate in Mozambique. <i>Hydrology and Earth System Sciences</i>, <i>26</i>(10), 2759–2778. <a href="https://doi.org/10.5194/hess-26-2759-2022">https://doi.org/10.5194/hess-26-2759-2022</a></div>
*  <div class="csl-entry"><i id="zotero|12028432/88UCGBRK"></i>Karatas, B. S., Akkuzu, E., Unal, H. B., Asik, S., &#38; Avci, M. (2009). Using satellite remote sensing to assess irrigation performance in Water User Associations in the Lower Gediz Basin, Turkey. <i>Agricultural Water Management</i>, <i>96</i>(6), 982–990. <a href="https://doi.org/10.1016/j.agwat.2009.01.010">https://doi.org/10.1016/j.agwat.2009.01.010</a></div>
*  <div class="csl-entry"><i id="zotero|12028432/9QBIFUHZ"></i>Karimi, P., Bongani, B., Blatchford, M., &#38; de Fraiture, C. (2019). Global Satellite-Based ET Products for the Local Level Irrigation Management: An Application of Irrigation Performance Assessment in the Sugarbelt of Swaziland. <i>Remote Sensing</i>, <i>11</i>(6), 705. <a href="https://doi.org/10.3390/rs11060705">https://doi.org/10.3390/rs11060705</a></div>
</div>
<!-- BIBLIOGRAPHY END -->


# Import Libraries


In [ ]:
library(terra)
library(ggplot2)
library(tidyterra)
library(dplyr)
library(scales)
library(readxl)

# Import Datasets

-   Seasonal AET, PCP (computed prior to this notebook using GEE python)
-   Boundary of scheme and Fields (from design document and digitizing the latest google earth image)
-   PET (ETo \* kc)
-   discharge data (measured from the scheme)


In [ ]:
fields <- vect("E:/PhD/Thesis/Irrigation Performance/Analysis/Data/Fields.shp")
boundary <- vect("E:/PhD/Thesis/Irrigation Performance/Analysis/Data/Boundary.shp") # nolint
et22 <- rast("E:/PhD/Thesis/Irrigation Performance/Result/Season Data/ETa22.tif") # nolint
et23 <- rast("E:/PhD/Thesis/Irrigation Performance/Result/Season Data/ETa23.tif") # nolint
pe22 <- 106.03 # Calculated from CHIRPS and USDA effective rainfall formula
pe23 <- 131.99 # Calculated from CHIRPS and USDA effective rainfall formula
etp22 <- 620.62
etp23 <- 608.92
wp22 <- rast("E:/PhD/Thesis/Irrigation Performance/Result/Season Data/WP22.tif")
wp23 <- rast("E:/PhD/Thesis/Irrigation Performance/Result/Season Data/WP23.tif")


In [ ]:
# Plot the fields
plot(fields, col = "lightblue", border = "black", lwd = 0.5)


In [ ]:
# print the number of Fields studied
nrow(fields)

# Statistics of Fields
summary(fields$Area_ha)


| Canal                     | Depth (m) | Width (m) | Area (m²) | Velocity (m/s) | Discharge (m³/s) | Discharge (m³) |
| ------------------------- | --------- | --------- | --------- | -------------- | ---------------- | -------------- |
| Main canal (Initial)      | 0.37      | 0.45      | 0.435     | 0.256          | 0.1215           | 1364688        |
| Main canal (Final)        | 0.32      | 0.41      | 0.398     | 0.188          | 0.0748           | 840154         |
| Secondary canal (initial) | 0.216     | 0.28      | 0.061     | 0.173          | 0.0105825552     | 825552         |
| Secondary canal (Final)   | 0.186     | 0.25      | 0.041     | 0.158          | 0.0064503194     | 503194         |


# Performance Indicators


## Equity

Equity is defined as the coefficients of variation (CV) of seasonal ETa in the area of interest.

-   It measures the evenness of the water supply in an irrigation scheme.
-   Note: CV of 0 to 10% is good, 10 to 25% is fair and CV > 25% is poor uniformity (Bastiaanssen et al., 1996)
    <br/>


### For Whole Scheme


In [ ]:
# Calculate the Max ETa for all fields (21/22)
et22_max <- zonal(et22, boundary, "max")
print(paste("Max ETa for 2021-22", et22_max))

# Calculate the mean ETa for all fields (21/22)
et22_mean <- zonal(et22, boundary, "mean")
print(paste("Mean ETa for 2021-22", et22_mean))

# Calculate the min ETa for all fields (21/22)
et22_min <- zonal(et22, boundary, "min")
print(paste("Min ETa for 2021-22", et22_min))

# Calculate the sd ETa for all fields (21/22)
et22_sd <- zonal(et22, boundary, "sd")
print(paste("SD ETa for 2021-22", et22_sd))

# Calculate the CV ETa for all fields (21/22)
et22_cv <- (et22_sd / et22_mean) * 100
print(paste("CV ETa for 2021-22", et22_cv))


In [ ]:
# Calculate the max ETa for all fields (22/23)
et23_max <- zonal(et23, boundary, "max")
print(paste("Mean ETa for 2022-23", et23_max))

# Calculate the mean ETa for all fields (22/23)
et23_mean <- zonal(et23, boundary, "mean")
print(paste("Mean ETa for 2022-23", et23_mean))

# Calculate the min ETa for all fields (22/23)
et23_min <- zonal(et23, boundary, "min")
print(paste("Min ETa for 2022-23", et23_min))

# Calculate the sd ETa for all fields (22/23)
et23_sd <- zonal(et23, boundary, "sd")
print(paste("SD ETa for 2022-23", et23_sd))

# Calculate the CV ETa for all fields (22/23)
et23_cv <- (et23_sd / et23_mean) * 100
print(paste("CV ETa for 2022-23", et23_cv))


### For Fields


In [ ]:
# Calculate the mean ETa for all fields (21/22)
et22_mean <- zonal(et22, fields, "mean")
# print(paste("Mean ETa for 2021-22", et22_mean)) # nolint

# Calculate the sd ETa for all fields (21/22)
et22_sd <- zonal(et22, fields, "sd")
# print(paste("SD ETa for 2021-22", et22_sd)) # nolint

# Calculate the CV ETa for all fields (21/22)
et22_cv <- (et22_sd / et22_mean) * 100
# print(paste("CV ETa for 2021-22", et22_cv)) # nolint

# Calculate the mean ETa for all fields (22/23)
et23_mean <- zonal(et23, fields, "mean")
# print(paste("Mean ETa for 2022-23", et23_mean)) # nolint

# Calculate the sd ETa for all fields (22/23)
et23_sd <- zonal(et23, fields, "sd")
# print(paste("SD ETa for 2022-23", et23_sd)) # nolint

# Calculate the CV ETa for all fields (22/23)
et23_cv <- (et23_sd / et23_mean) * 100
#print(paste("CV ETa for 2022-23", et23_cv)) # nolint


In [ ]:
# Statsistics
summary(et22_mean)
summary(et23_mean)


In [ ]:
summary(et22_sd)
summary(et23_sd)


In [ ]:
summary(et22_cv)
summary(et23_cv)


## Adequacy

The adequacy indicator is used to measure the sufficiency of water delivery to a known command area, as well as the reduction in ETa

Adequacy can be calculated as the ratio of actual evapotranspiration (ETa) to potential evapotranspiration (ETp).

-   It measures the adequacy of the water supply in an irrigation scheme.
-   Note: Adequacy of 0.8 to 1.0 is good, 0.6 to 0.8 is fair, and Adequacy < 0.6 is poor adequacy (Bastiaanssen et al., 1996)

$$Adequacy = \frac{ET_a}{ET_p}$$

It can also be calculated as relative water supply:
$$RWS=\frac{V_c+P_g}{ET_p}$$

Deplted Fraction:
$$DF=\frac{ET_a}{P_g + V_c}$$


In [ ]:
# Adequacy 2021-22
ret22 <- (et22 / etp22)

# Plot it
plot(ret22)

# Save it as geotiff
#writeRaster(ret22, "E:/PhD/Thesis/Irrigation Performance/Result/Season Data/Adequacy22.tif", overwrite = TRUE) # nolint


In [ ]:
# Adequacy 2022-23
ret23 <- (et23 / etp23)

# Plot it
plot(ret23)

# Save it as geotiff
#writeRaster(ret23, "E:/PhD/Thesis/Irrigation Performance/Result/Season Data/Adequacy23.tif", overwrite = TRUE) # nolint


In [ ]:
# Stack the rasters together
rasters <- c(ret22, ret23)

names(rasters) <- c("2021/22", "2022/23")

# Plot
p <- ggplot() +
  geom_spatraster(data = rasters) +
  facet_wrap(~lyr, ncol = 2) +
  scale_fill_whitebox_c(
    palette = "deep", na.value = "white",
    n.breaks = 8,
    guide = guide_legend(reverse = TRUE, keyheight = unit(1.75, "cm"))
  ) + # nolint
  labs(fill = "RET") +
  theme_bw() +
  theme(
    axis.text.y = element_text(angle = 90, vjust = 0.5, size = 16),
    axis.text.x = element_text(size = 16),
    strip.text = element_text(size = 16),
    panel.grid = element_blank(),
    text = element_text(size = 16)
  ) +
  scale_x_continuous(labels = function(x) format(x, trim = TRUE, scientific = FALSE), breaks = c(37.37, 37.39, 37.41)) + # nolint
  scale_y_continuous(labels = function(y) format(y, trim = TRUE, scientific = FALSE), breaks = c(10.368, 10.383, 10.398)) # nolint
p

# Save the plot as a JPEG with A4 landscape size and 900 DPI
#ggsave("Adequacy_Plot.jpg", plot = p, width = 14, height = 6, units = "in", dpi = 900) # nolint


In [ ]:
# Zonal on Adequacy
ret22_mean <- zonal(ret22, fields, "mean")
print(paste("Mean Adequacy for 2021-22", ret22_mean))

ret23_mean <- zonal(ret23, fields, "mean")
print(paste("Mean Adequacy for 2022-23", ret23_mean))


In [ ]:
# Statistics
summary(ret22_mean)
summary(ret23_mean)


## Productivity


### Crop Water Productivity (CWP)

It's can be defined as the ratio of biomass production to the amount of water used (ET). Saying it differently, it is the amount of biomass produced per unit of water evaporated.

$$
\text{CWP} = \text{Biomass Water Productivity} \times \text{Harvest Index}
$$

-   Harvest index of wheat (Kekeba variety at Debre Elias) is 0.3008 (Sewunet et al., 2021)


In [ ]:
# Calculate the mean Biomass WP for all fields (21/22)
cwp22 <- wp22 * 0.3008
cwp22_mean <- zonal(cwp22, fields, "mean")

# Print the result
print(paste("2021-22 WP", cwp22_mean))

# Calculate the mean Biomass WP for all fields (22/23)
cwp23 <- wp23 * 0.3008
cwp23_mean <- zonal(cwp23, fields, "mean")

# Print the result
print(paste("2022-23 WP", cwp23_mean))


In [ ]:
# Statistics
summary(cwp22_mean)
summary(cwp23_mean)


### Land Prodctivity


In [ ]:
production <- read.csv("E:/PhD/Thesis/Irrigation Performance/Analysis/Farmers Production.csv") # nolint
head(production)
str(production)

In [ ]:
sort(production$Production_2014, decreasing = FALSE)
sort(production$Production_2015, decreasing = FALSE)

In [ ]:
summary(production)
